# CSE 5819 Assignment #3

**by: Aayushi Verma (uef24001)**

*due: Fri 9/20/24*

**Part 2 [Programming] (20 pts)** 

**In HW2, you have already loaded the Seaborn Iris dataset
(The dataset has four features: sepal_length, sepal_width, petal_length, petal_width, and
one target: species. It has three flower species: setosa, versicolor, and virginica)
Use the provide cvxpy_demo.ipynb in HuskyCT to solve least squares. In other words,
we will consider the hypothesis class that contains all linear functions ℎ(𝑥) = 𝑤𝑇𝑥 + 𝑏,
and the squared loss**

$$
L_{sq} = \frac{1}{n} \sum_{i=1}^n (h(x_i)-y_i)^2
$$

**Note that this is very different from the above question 1 because we are now considering
a different hypothesis class. (Also note that for the Iris data, the squared loss is not the
best loss because the target variable “species” is a category variable, but for this problem,
we just simply treat it as a regression problem, so we set setosa = 1, versicolor = 2, and
virginica = 3.)**

In [1]:
# importing packages
import seaborn as sns
import pandas as pd
import numpy as np
import cvxpy as cvx

First we load the dataset using the Seaborn library. 

**Note to TA:** I was having issues loading the dataset from Seaborn (ssl certificate error) so instead downloaded a local csv of the dataset. For the submission of this assignment on Google Colab, I will comment out my code which uses the local download of the datatset and keep the original line of code, and hope that it works on your end.

In [2]:
# downloading the dataset from the Seaborn library
iris = sns.load_dataset('iris')

# mapping species to numeric labels
species_mapping = {'setosa': 1, 'versicolor': 2, 'virginica': 3}
iris['species'] = iris['species'].map(species_mapping)

In [3]:
# # loading the iris dataset from my locally-downloaded version
# iris = pd.read_csv('iris.csv')
# # renaming the columns to match the Seaborn version
# iris.rename(columns={
#     'SepalLengthCm': 'sepal_length',
#     'SepalWidthCm': 'sepal_width',
#     'PetalLengthCm': 'petal_length',
#     'PetalWidthCm': 'petal_width',
#     'Species': 'species'
# }, inplace=True)

# # dropping the Id column since Seaborn version doesn't have it
# iris.drop(columns=['Id'], inplace=True)

# # mapping species to numeric labels
# species_mapping = {'Iris-setosa': 1, 'Iris-versicolor': 2, 'Iris-virginica': 3}
# iris['species'] = iris['species'].map(species_mapping)

In [4]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


Now that we have converted the Iris dataset into a numerical dataset, we can proceed by making `X` and `y` matrices.

In [5]:
X = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [6]:
y = iris['species'].values
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

We will now instantiate some necessary variables to use for the problem. We start with getting some dimensional numbers.

In [7]:
# n = mumber of data points (rows), d = number of features (columns)
n, d = X.shape  
print(n, d)

150 4


In [8]:
# w = weight vector of length d, set up as a Variable object
w = cvx.Variable(d) 
w

Variable((4,), var1)

In [9]:
# b = bias vector, set up as an empty Variable object
b = cvx.Variable() 
b

Variable((), var2)

With our variables now defined, we can set up the squared loss function such that it is minimized. Remember that we want:
$$
min (L_{sq}) = \frac{1}{n} \sum_{i=1}^n (h(x_i)-y_i)^2 = \frac{1}{n} \sum_{i=1}^n (w_i^T X_i + b -y_i)^2
$$

In [10]:
sq_loss = cvx.Minimize(1/n * cvx.sum_squares(X @ w + b - y))
sq_loss

Minimize(Expression(CONVEX, NONNEGATIVE, ()))

Finally we set the problem to solve.

In [11]:
problem = cvx.Problem(sq_loss)
problem.solve()

np.float64(0.046385088311226945)

Obtaining the solution returned by the `cvxpy` solver, we have:

In [12]:
print("Optimal weights (w):", w.value)
print("Optimal intercept (b):", b.value)

Optimal weights (w): [-0.10974146 -0.04424045  0.22700138  0.60989412]
Optimal intercept (b): 1.192083994828139
